# 4D scan

In [ ]:
import sys
import os
from os.path import join
from pprint import pprint
import importlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
import pandas as pd
import h5py
from scipy import ndimage
import proplot as pplt

sys.path.append('..')
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

## Load data 

In [ ]:
datadir = '/Diagnostics/Data/Measurements/transverse4d/2022-01-04/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220104192006-transverse4d'
file = h5py.File(join(datadir, filename + '-preproc.h5'), 'r')
data = file['scalar']
pprint(data.dtype.fields)

In [ ]:
slit_dict = {
    'xp': {
        'center': 13.5,
        'distance': 10.0,
        'steps': 20,
    },
    'x': {
        'center': 13.0,
        'distance': 15.0,
        'steps': 20,
    },
    'yp': {
        'center': 17.0,
        'distance': 7.0,
        'steps': 20,
    },
    'y': {
        'center': 14.5,
        'distance': 22.0,
        'steps': 20,
    },
}
keys = list(slit_dict)
M = np.identity(4)
M[keys.index('xp'), keys.index('x')] = 0.8
M[keys.index('yp'), keys.index('y')] = 0.7
Minv = np.linalg.inv(M)

center = np.array([slit_dict[key]['center'] for key in keys])
distance = np.array([slit_dict[key]['distance'] for key in keys])
steps = np.array([slit_dict[key]['steps'] for key in keys])

In [ ]:
x1 = data['x_PositionSync'].copy()
x2 = data['xp_PositionSync'].copy()
y1 = data['y_PositionSync'].copy()
y2 = data['yp_PositionSync'].copy()
points = np.vstack([x2, x1, y2, y1]).T
points_n = utils.apply(Minv, points - center)
points_nn = points_n / (0.5 * distance)

In [ ]:
dims = ['x2', 'x1', 'y2', 'y1']
for _points, title in zip((points, points_n, points_nn), ('true', 'upright', 'upright + scaled')):
    fig, axes = pplt.subplots(ncols=4, nrows=4, figwidth=6.0, spanx=False, spany=False)
    axes.format(suptitle=title)
    for i in range(4):
        for j in range(4):
            axes[i, j].scatter(_points[:, j], _points[:, i], c='black', ec='None', s=2)
        axes[i, 0].format(ylabel=dims[i])
        axes[-1, i].format(xlabel=dims[i])
    plt.show()

In [ ]:
signal = data['cam06_Integral']
thresh = 0.001 * np.max(signal)
idx, = np.where(signal >= thresh)

fig, ax = pplt.subplots(figsize=(10.0, 2.5))
ax.plot(signal + 1.0, color='lightgray', lw=0, marker='.', ms=2, ec='None')
ax.plot(idx, signal[idx] + 1.0, color='black', lw=0, marker='+', ms=0.5)
ax.format(yscale='log')
plt.show()

In [ ]:
radii = np.sqrt(np.sum(np.square(points_nn), axis=1))

bins = 'auto'
for yscale in [None, 'log']:
    with pplt.rc.context(legendfontsize='medium'):
        fig, ax = pplt.subplots(figsize=(3, 1.85))
        ax.hist(radii, bins=bins, label='all', color='lightgrey')
        ax.hist(radii[idx], bins=bins, label='above thresh', color='black')
        ax.format(ylabel='num. points', xlabel='radius', yscale=yscale)
        ax.legend(ncols=2, loc='top', framealpha=0)
        plt.savefig(f'_output/radii_yscale{yscale}.png')
        plt.show()

In [ ]:
rmax = np.max(radii[idx])
rmax

In [ ]:
frac = np.count_nonzero(radii <= rmax) / len(radii)
frac

In [ ]:
frac_signal = float(len(signal[idx])) / len(signal)
print(f'{frac_signal:.3f}')

In [ ]:
volume_ratio = utils.volume_sphere(n=4, r=rmax) / utils.volume_box(n=4, r=1.0)
print(volume_ratio)

In [ ]:
_points = points_nn
_rmax = rmax

fig, axes = pplt.subplots(ncols=4, nrows=4, figwidth=5.0, spanx=False, spany=False)
# axes.format(suptitle=f'fraction (r < {rmax:.2f}) = {frac:.2f}',
#             suptitle_kw=dict(fontweight='normal'))
for i in range(4):
    for j in range(4):
        ax = axes[i, j]
        ax.scatter(_points[:, j], _points[:, i], c='lightgray', ec='None', s=0.5)
        ax.scatter(_points[idx, j], _points[idx, i], c='black', ec='None', s=0.5)
        if i != j:
            ax.add_patch(patches.Ellipse((0.0, 0.0), 2.0 * _rmax, 2.0 * _rmax, color='red', fill=False))
    axes[i, 0].format(ylabel=dims[i])
    axes[-1, i].format(xlabel=dims[i])
plt.savefig('_output/bounding_ellipse.png')
plt.show()